<br><br>
# Explainable k-means

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

## Define funcs
I did not create Class of their algorithm, so this is a little experiment for me.

In [2]:
#ソートと動的計画法によって，最適な分割を求める
def optimal_threshold_2means(X):
    bests_split = {'cost':np.inf,'coordinate':None,'threshold':None}
    data_num = X.shape[0]
    data_dimentions = X.shape[1]
    u = np.sum(X*X)
    
    for i in range(data_dimentions):
        s = np.zeros(data_dimentions)
        r = np.sum(X,axis=0)
        ith_sorted_X = X[X[:,i].argsort(), :]
        for j,data in enumerate(X[:-1]):
            s += data
            r -= data
            cost = u - np.sum(s*s)/(j+1) -np.sum(r*r)/(data_num-j-1)
            #print(cost)
            if cost < bests_split['cost'] and X[j][i]!=X[j+1][i]:
                bests_split['cost'] = cost
                bests_split['coordinate'] = i
                bests_split['threshold'] = data[i]
            
    return bests_split

#最適な分割に基づいてクラスタリング
def clustering_2means_by_tree(bests_split,X):
    cluster = np.ones(X.shape[0])
    for i,data in enumerate(X):
        if(data[bests_split['coordinate']]>bests_split['threshold']):
            cluster[i] = 0
    return cluster

#得られた分割の中心座標を求める
def get_mean(X,approx_labels):
    res=[]
    for k in range(len(np.unique(approx_labels))):
        n = 0
        mean = np.zeros(X.shape[1])
        for i,data in enumerate(X):
            if(approx_labels[i]==k):
                mean+=data
                n+=1
        res.append(mean/n)
    return np.array(res)

#近似の比率を計算する，論文ではkmeansの場合，上界は4である．
def approx_score(approx_labels,kmeans_model,X):
    kmeans_cost = 0
    kmeans_label = kmeans_model.labels_
    kmeans_centers = kmeans_model.cluster_centers_
    for i,data in enumerate(X):
        kmeans_cost += np.sum((data-kmeans_centers[kmeans_label[i]])*(data-kmeans_centers[kmeans_label[i]]))
    
    approx_cost = 0
    mean = get_mean(X,approx_labels)
    for k in range(kmeans_model.n_clusters):
        for i,data in enumerate(X):
            if(approx_labels[i]==k):
                approx_cost += np.sum((data-mean[k])*(data-mean[k]))
    print(kmeans_cost)
    print(approx_cost)
    return approx_cost/kmeans_cost

## Two Datasets
First is from this good article.  
Second is from uci repo and I get the code from this good article.

In [4]:
# First dataset
# 生徒の国語・数学・英語の各得点を配列として与える
X = np.array([
        [  80,  85, 100 ],
        [  96, 100, 100 ],
        [  54,  83,  98 ],
        [  80,  98,  98 ],
        [  90,  92,  91 ],
        [  84,  78,  82 ],
        [  79, 100,  96 ],
        [  88,  92,  92 ],
        [  98,  73,  72 ],
        [  75,  84,  85 ],
        [  92, 100,  96 ],
        [  96,  92,  90 ],
        [  99,  76,  91 ],
        [  75,  82,  88 ],
        [  90,  94,  94 ],
        [  54,  84,  87 ],
        [  92,  89,  62 ],
        [  88,  94,  97 ],
        [  42,  99,  80 ],
        [  70,  98,  70 ],
        [  94,  78,  83 ],
        [  52,  73,  87 ],
        [  94,  88,  72 ],
        [  70,  73,  80 ],
        [  95,  84,  90 ],
        [  95,  88,  84 ],
        [  75,  97,  89 ],
        [  49,  81,  86 ],
        [  83,  72,  80 ],
        [  75,  73,  88 ],
        [  79,  82,  76 ],
        [ 100,  77,  89 ],
        [  88,  63,  79 ],
        [ 100,  50,  86 ],
        [  55,  96,  84 ],
        [  92,  74,  77 ],
        [  97,  50,  73 ],
        ])

In [5]:
#second is from uci
cust_df = pd.read_csv("http://pythondatascience.plavox.info/wp-content/uploads/2016/05/Wholesale_customers_data.csv")
del(cust_df['Channel'])
del(cust_df['Region'])
cust_array = np.array([cust_df['Fresh'].tolist(),
                       cust_df['Milk'].tolist(),
                       cust_df['Grocery'].tolist(),
                       cust_df['Frozen'].tolist(),
                       cust_df['Milk'].tolist(),
                       cust_df['Detergents_Paper'].tolist(),
                       cust_df['Delicassen'].tolist()
                       ], np.int32)
cust_array = cust_array.T
cust_array.shape

(440, 7)

## First data and k=2 

In [9]:
#K-meansクラスタリングをおこなう
kmeans_model = KMeans(n_clusters=2, random_state=10).fit(X)
#分類先となったラベルを取得する
labels = kmeans_model.labels_

#提案手法による近似アルゴリズムで取得する
bests_split = optimal_threshold_2means(X)
approx_labels = clustering_2means_by_tree(bests_split,X)
print(bests_split)
print(approx_labels)

{'cost': 15800.027777777781, 'coordinate': 0, 'threshold': 49}
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
approx_score(approx_labels,kmeans_model,X) #近似アルゴリズムのコスト ➗ k-meansのコスト

11316.030172413793
15320.328571428567


1.3538607036217036

## Second data and k=2

In [6]:
#K-meansクラスタリングをおこなう
kmeans_model = KMeans(n_clusters=2, random_state=10).fit(cust_array)
#分類先となったラベルを取得する
labels = kmeans_model.labels_

#提案手法による近似アルゴリズムで取得する
bests_split = optimal_threshold_2means(cust_array)
approx_labels = clustering_2means_by_tree(bests_split,cust_array)
print(bests_split)

{'cost': 82634911605.12671, 'coordinate': 0, 'threshold': 9198}


In [7]:
approx_score(approx_labels,kmeans_model,cust_array) #近似アルゴリズムのコスト ➗ k-meansのコスト

132340344661.33641
147714287599.63416


1.116169736278383

<br><br>
## IMM Algorithm
I'm working on this....

In [11]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None,condition=None):
        self.val = val
        self.left = left
        self.right = right
        self.condition = (0,0)

In [12]:
def mistake():
    
def build_tree(X,labels,centers):
    node = TreeNode()
    if(len(np.unique(labels))==1):
        node.val = labels[-1]
        return node
    
    l=[]
    r=[]
    for i in enumerate(centers):
        
    
    node.condition = (i,threshold)
    node.left = build_tree()
    node.right = build_tree()
    return node

IndentationError: expected an indented block (<ipython-input-12-f4c1a8ed870b>, line 3)

In [13]:
#IMM procedure
kmeans_model = KMeans(n_clusters=3, random_state=10).fit(features)
centers = kmeans_model.cluster_centers_
labels = kmeans_model.labels_

build_tree(X,labels,centers)

NameError: name 'features' is not defined